<a href="https://colab.research.google.com/github/alexpin/Curso-Python/blob/master/EDA/3_enriquecimiento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%matplotlib inline
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive') 

Mounted at /content/drive


In [ ]:
compras_df = pd.read_csv('/content/drive/My Drive/Proyectos de datos/notebooks/data/clean_compras.csv')

In [ ]:
compras_df.dtypes

NUM BEN           int64
BENEFICIARIO     object
RFC              object
FACTURA          object
CONTRATO         object
CONCEPTO         object
TIPO             object
# OP            float64
FECHA            object
IMPORTE         float64
SHEET            object
dtype: object

## Empezamos con una expansion de datos que tenemos, fecha por ejemplo

In [ ]:
compras_df['FECHA'] = pd.to_datetime(compras_df['FECHA'])

In [ ]:
compras_df['DIA'] = compras_df['FECHA'].apply(lambda x: x.day)

In [ ]:
compras_df['MES'] = compras_df['FECHA'].apply(lambda x: x.month)

In [ ]:
compras_df['ANO'] = compras_df['FECHA'].apply(lambda x: x.year)

## Proyectemos unas variables contra otras

In [ ]:
beneficiario_sales = compras_df['BENEFICIARIO'].value_counts().to_dict()

In [ ]:
list(beneficiario_sales.keys())[0:5]

['CAMARA MEXICANA DE LA INDUSTRIA DE LA CONSTRUCCION',
 'AUTOKAM REGIOMONTANA,S.A.DE C.V.',
 'CAR ONE MONTERREY, S.A. DE C.V.',
 'PEREZ RIOS RAMIRO',
 'S.I.M.E.P.R.O.D.E.']

In [ ]:
compras_df['BENEFICIARIO_SALES'] = compras_df['BENEFICIARIO'].apply(lambda x: beneficiario_sales[x]) 

In [ ]:
sheet_mean_compras = compras_df.groupby('SHEET')['IMPORTE'].mean()

In [ ]:
sheet_mean_compras

SHEET
COMUNICACION              109705.343662
CONTRATISTAS Y FDO FED    474566.765787
GASTOS VARIOS             881659.731797
SERV PPROF                229592.166667
SERV. PERS.                59623.405217
Name: IMPORTE, dtype: float64

In [ ]:
sheet_mean_compras.to_dict()

{'COMUNICACION': 109705.34366197183,
 'CONTRATISTAS Y FDO FED': 474566.76578729285,
 'GASTOS VARIOS': 881659.7317971016,
 'SERV PPROF': 229592.16666666666,
 'SERV. PERS.': 59623.40521739131}

In [ ]:
compras_df['IMPORTE_GASTO_PROMEDIO'] = compras_df['SHEET'].apply(lambda x: sheet_mean_compras[x]) 

## Traernos informacion externa, enriquecer los datos usando una tool de paginas de linkedin

In [ ]:
pd.DataFrame(compras_df['BENEFICIARIO'].unique()).to_csv('/content/drive/My Drive/Proyectos de datos/notebooks/data/unique_beneficiarios.csv', index=None)

In [ ]:
#Cargar
linkedin_mty = pd.read_csv('/content/drive/My Drive/Proyectos de datos/notebooks/data/linkedin_mty.csv')

In [ ]:
has_site = {}
for i, ele in linkedin_mty.fillna(True).iterrows():
    has_site[ele['query']] = True if ele['error'] == True else False

In [ ]:
compras_df['HAS_LINKEDIN_SITE'] = compras_df['BENEFICIARIO'].apply(lambda x: has_site[x.replace('  ', ' ')])

In [ ]:
compras_df['HAS_LINKEDIN_SITE'] 

0       True
1       True
2       True
3       True
4       True
        ... 
1535    True
1536    True
1537    True
1538    True
1539    True
Name: HAS_LINKEDIN_SITE, Length: 1540, dtype: bool

In [ ]:
compras_df['HAS_LINKEDIN_SITE'].value_counts()

True     1392
False     148
Name: HAS_LINKEDIN_SITE, dtype: int64

In [ ]:
compras_df.to_csv('/content/drive/My Drive/Proyectos de datos/notebooks/data/compras_df_enriched.csv', index=None)